In [ ]:
import os
import glob
import cv2
import numpy as np
import ast
from pprint import pprint
import operator as op
import ffmpeg
from typing import Dict, Union
import matplotlib.pyplot as plt
import imagehash
from skimage.metrics import structural_similarity

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
#files = glob.glob(f'{os.path.join(DATA_FOLDER, "example_videos_left")}/*.mp4')
files = glob.glob("/media/jakki/Seagate_Expansion_Drive/keparoi/2021/matsi_05082021/right/*.MOV")

In [ ]:
files

In [ ]:
def calculate_frame_similarity_thresholding(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    #blur_gray1 = cv2.GaussianBlur(gray1, (3, 3), 0)
    #blur_gray2 = cv2.GaussianBlur(gray2, (3, 3), 0)

    frame_height = gray1.shape[0]
    frame_width = gray1.shape[1]

    n_pixels = frame_height * frame_width

    diff_frame = cv2.absdiff(gray1, gray2)

    thresh_frame = cv2.threshold(src=diff_frame, thresh=50, maxval=255, type=cv2.THRESH_BINARY)[1]
    
    difference = np.sum(thresh_frame)
    difference_per_pixel = difference / n_pixels

    if difference_per_pixel < 1:
        plt.imshow(thresh_frame)
        plt.show()
    
    return difference_per_pixel

In [ ]:
def calculate_frame_similarity_advanced(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    blur_gray1 = cv2.GaussianBlur(gray1, (3, 3), 0)
    blur_gray2 = cv2.GaussianBlur(gray2, (3, 3), 0)

    mse = np.mean((gray1 - gray2) ** 2)
    mse_score = mse / 255
        
    return mse_score

In [ ]:
def calculate_frame_similarity(frame1, frame2):
    im1_hash = imagehash.average_hash(frame1)
    im1_hash = imagehash.average_hash(frame2)
    
    return im1_hash - im2_hash

In [ ]:
def calculate_SSIM(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Compute SSIM between two images
    score, diff = structural_similarity(gray1, gray2, full=True)
    return score

In [ ]:
calculate_diff = calculate_frame_similarity_thresholding

In [ ]:
allowed_operators = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
                     ast.Div: op.truediv, ast.Pow: op.pow, ast.BitXor: op.xor,
                     ast.USub: op.neg}

def eval_expr(expr):
    """
    >>> eval_expr('2^6')
    4
    >>> eval_expr('2**6')
    64
    >>> eval_expr('1 + 2*3**(4^5) / (6 + -7)')
    -5.0
    """
    return eval_(ast.parse(expr, mode='eval').body)

def eval_(node):
    if isinstance(node, ast.Num):  # <number>
        return node.n
    elif isinstance(node, ast.BinOp):  # <left> <operator> <right>
        return allowed_operators[type(node.op)](eval_(node.left), eval_(node.right))
    elif isinstance(node, ast.UnaryOp):  # <operator> <operand> e.g., -1
        return allowed_operators[type(node.op)](eval_(node.operand))
    else:
        raise TypeError(node)

def get_video_info(video_path) -> Dict[str, Union[int, float, str]]:
    probe = ffmpeg.probe(video_path)
    file_path = str(probe['format']['filename'])
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    width = int(video_stream['width'])
    height = int(video_stream['height'])
    duration = float(probe['format']['duration'])
    frame_rate = round(eval_expr(video_stream['avg_frame_rate']))
    # Frame rate removed in notebook because cannot import relative local utils
    
    return {
        "file_path" : file_path,
        "frame_height": height,
        "frame_width": width,
        "frame_rate": frame_rate,
        "duration": duration
    }

In [ ]:
def get_last_frame(capture, duration):
    # Dirty hack to get to last 3 seconds to avoid reading to whole video file
    capture.set(cv2.CAP_PROP_POS_MSEC,(duration-3)*1000)
    last_frame = None
    while True:
        ret, tmp_frame = capture.read()
        if not ret:
            break
        last_frame = tmp_frame
        
    success = last_frame is not None
    return success, last_frame

In [ ]:
def add_edge(mapping, node1, node2):
    if node1 in mapping:
        mapping[node1].append(node2)
    else:
        mapping[node1] = [node2]


In [ ]:
for file in files:
    print(file)

In [ ]:
frames = {}

for file_path in files:
    
    video_info = get_video_info(file_path)
    duration = video_info['duration']
    print(duration)
    
    capture = cv2.VideoCapture(file_path)
    
    # Read first frame
    capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret_first, first_frame = capture.read()

    # Read last frame
    ret_last, last_frame = get_last_frame(capture, duration)
    
    if ret_first is False or ret_last is False:
        print(f"Failed to read frame from video {file_path}, first frame: {ret_first}, last frame: {ret_last}")
    
    frames[file_path] = {
        'first_frame': first_frame,
        'last_frame': last_frame
    }

In [ ]:
n_files = len(frames)
similarity_matrix = {}

for i, pair1 in enumerate(frames.items()):
    for j, pair2 in enumerate(frames.items()):
        if i < j:
            key1, values1 = pair1
            key2, values2 = pair2
                                    
            first_vs_last = calculate_diff(values1['first_frame'], values2['last_frame'])      
            last_vs_first = calculate_diff(values1['last_frame'], values2['first_frame'])
            
            similarity_matrix[(key1, key2)] = [first_vs_last, last_vs_first]

In [ ]:
# Build similarity matrix
n_files = len(files)
similarity_matrix = [[0.0] * (2 * n_files) for _ in range(2 * n_files)]

for i, file1 in enumerate(files):
    frames1 = frames[file1]
    for j, file2 in enumerate(files):
        frames2 = frames[file2]

        # Only calculate upper triangle to avoid duplicates
        if i < j:
            # Compare first frame of file1 to both frames of file2
            similarity_matrix[i*2][j*2] = calculate_diff(
                frames1['first_frame'], frames2['first_frame'])
            similarity_matrix[i*2][j*2+1] = calculate_diff(
                frames1['first_frame'], frames2['last_frame'])

            # Compare last frame of file1 to both frames of file2
            similarity_matrix[i*2+1][j*2] = calculate_diff(
                frames1['last_frame'], frames2['first_frame'])
            similarity_matrix[i*2+1][j*2+1] = calculate_diff(
                frames1['last_frame'], frames2['last_frame'])

            # Mirror the values to lower triangle
            similarity_matrix[j*2][i*2] = similarity_matrix[i*2][j*2]
            similarity_matrix[j*2][i*2+1] = similarity_matrix[i*2][j*2+1]
            similarity_matrix[j*2+1][i*2] = similarity_matrix[i*2+1][j*2]
            similarity_matrix[j*2+1][i*2+1] = similarity_matrix[i*2+1][j*2+1]

# Print similarity matrix
print("\nSimilarity Matrix:")
header = []
for file in files:
    name = os.path.basename(file)
    name = name.replace("keparoicam_left_", "")
    name = name.replace(".mp4", "")
    header.extend([f"{name}_first", f"{name}_last"])

print("    " + "  ".join(f"{h:>10}" for h in header))
for i in range(2 * n_files):
    row_name = header[i]
    values = [f"{x:10.4f}" for x in similarity_matrix[i]]
    print(f"{row_name:10}: {' '.join(values)}")

# Find N-1 best matches (excluding diagonal and duplicates)
matches = set()  # Use set to avoid duplicates
for i in range(2 * n_files):
    for j in range(2 * n_files):
        # Skip diagonal elements and lower triangle
        if i//2 < j//2:  # Different files, upper triangle only
            print(f"Add: {i}, {j}")
            matches.add((i, j, similarity_matrix[i][j]))

print(matches)
            
# Sort by similarity score
matches = sorted(matches, key=lambda x: x[2])
best_matches = matches[:n_files-1]
print(best_matches)

video_end_to_start_mapping = {}
for i, j, score in best_matches:
    if i % 2 == 0:
        file1 = files[j//2]
        file2 = files[i//2]
    else:
        file1 = files[i//2]
        file2 = files[j//2]
    print(f"Best match: {os.path.basename(file1)} -> {os.path.basename(file2)} (score: {score:.4f})")
    add_edge(video_end_to_start_mapping, file1, file2)
    
print(video_end_to_start_mapping)

In [ ]:
def check_linking_valid(mapping):
    """We want to check that
        a) there is only one root node
        b) the chain is not broken e.g. each node links to another node
        c) there is only one child node that doesn't exist in keys
        d) each node has exactly one parent except root node and each node has exactly one child
    """

    childs = [value[0] for value in mapping.values()]
    keys = list(mapping.keys())

    # a)
    root_nodes = list(set(keys) - set(childs))
    if len(root_nodes) != 1:
        return False

    # b)
    visited_nodes = []
    last_node = root_nodes[0]
    next_node = None
    for _ in range(len(mapping)):
        next_nodes = mapping[last_node]
        if len(next_nodes) != 1:
            logger.error("Cannot determine video linking")
            logger.debug(f"Video mapping: {mapping}")
            sys.exit(1)
        next_node = next_nodes[0]
        visited_nodes.append(next_node)
        last_node = next_node

    if len(visited_nodes) != len(mapping):
        return False

    # c)
    leafs = list(set(childs) - set(keys))
    if len(leafs) != 1:
        return False

    return True


In [ ]:
check_linking_valid(video_end_to_start_mapping)